# 手牌回放查看器

回放已记录的牌局，支持交互式控制。

In [ ]:
from holdem_lab import (
    parse_cards, format_cards,
    GameState, Street,
    EventLog, EventType, HandReplayer,
)
try:
    from ipywidgets import interact, IntSlider, Output
    HAS_WIDGETS = True
except ImportError:
    HAS_WIDGETS = False
    print("注意: ipywidgets 未安装。交互式控件已禁用。")
    print("安装命令: pip install ipywidgets")

## 生成示例牌局

In [ ]:
# 运行一局牌并启用日志记录
game = GameState(num_players=3, seed=42, log_events=True)

# 手动逐街进行以记录权益
game.deal_hole_cards()
game.calculate_equity(num_simulations=1000)  # 记录翻前权益

game.deal_flop()
game.calculate_equity(num_simulations=1000)  # 记录翻牌权益

game.deal_turn()
game.calculate_equity(num_simulations=1000)  # 记录转牌权益

game.deal_river()
result = game.resolve()

print(f"牌局完成！获胜者: 玩家 {result.winners[0]}")
print(f"获胜牌型: {result.winning_hand.describe()}")
print(f"记录事件总数: {len(game.event_log)}")

## 查看所有事件

In [ ]:
# 显示所有事件
log = game.event_log
print("事件日志:")
print("=" * 60)

for i, event in enumerate(log):
    print(f"[{i:2}] {event.event_type.value:15} | {event.data}")

## 手动逐步回放

In [ ]:
# 创建回放器并逐步执行
replayer = HandReplayer(log)

def display_state(state):
    """格式化显示手牌状态。"""
    if state is None:
        print("无状态")
        return
    
    print(f"步骤 {state.step}: {state.event.event_type.value}")
    print("-" * 40)
    
    # 显示底牌
    if state.hole_cards:
        print("底牌:")
        for player, cards in state.hole_cards.items():
            cards_str = format_cards(cards) if cards else "?"
            print(f"  玩家 {player}: {cards_str}")
    
    # 显示公共牌
    if state.board:
        print(f"公共牌: {format_cards(state.board)}")
    
    # 显示权益
    if state.equities:
        print("权益:")
        for player, eq in state.equities.items():
            print(f"  玩家 {player}: {eq:.1%}")
    
    # 显示获胜者
    if state.winners is not None:
        print(f"获胜者: {state.winners}")
    
    print()

In [ ]:
# 回放所有状态
replayer.reset()
print("完整牌局回放")
print("=" * 60)

states = replayer.get_all_states()
for state in states:
    display_state(state)

## 交互式滑块 (需安装 ipywidgets)

In [ ]:
if HAS_WIDGETS:
    output = Output()
    
    def show_step(step):
        output.clear_output(wait=True)
        with output:
            state = replayer.goto_step(step)
            display_state(state)
    
    print("使用滑块浏览牌局:")
    interact(show_step, step=IntSlider(min=0, max=len(log)-1, step=1, value=0))
    display(output)
else:
    print("交互式滑块需要安装 ipywidgets。")
    print("请使用上方的手动回放功能。")

## 保存和加载牌局

In [ ]:
# 保存牌局日志
import tempfile
import os

# 保存到临时文件
temp_dir = tempfile.mkdtemp()
save_path = os.path.join(temp_dir, "sample_hand.json")

log.save(save_path)
print(f"已保存到: {save_path}")

# 重新加载
loaded_log = EventLog.load(save_path)
print(f"已加载 {len(loaded_log)} 个事件")

In [ ]:
# 验证加载的日志
loaded_replayer = HandReplayer(loaded_log)
loaded_states = loaded_replayer.get_all_states()

print("加载的牌局摘要:")
print(f"  事件总数: {len(loaded_log)}")
print(f"  牌局 ID: {loaded_log.hand_id}")

# 找到摊牌事件
for state in loaded_states:
    if state.event.event_type == EventType.SHOWDOWN:
        print(f"  获胜者: 玩家 {state.winners}")
        break

## 运行多局牌并比较

In [ ]:
# 运行多局牌并收集统计数据
num_hands = 100
winners_count = {0: 0, 1: 0, 2: 0}

for seed in range(num_hands):
    game = GameState(num_players=3, seed=seed, log_events=False)
    result = game.run_to_showdown()
    for w in result.winners:
        winners_count[w] += 1 / len(result.winners)  # 平分平局

print(f"{num_hands} 局牌的结果:")
for player, wins in winners_count.items():
    print(f"  玩家 {player}: {wins:.1f} 次获胜 ({wins/num_hands*100:.1f}%)")